# Plan
- Get line by using cv2 to detect lines and connections between nodes
- nodes will always be in the same locations, ROI will not change
- Find contours is too complex to use, need to simplify to reduce room of errors

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
from utils import preprocess
from statistics import mean

In [ ]:
def canny(img, th_1=50, th_2=200):
    img_canny = cv.Canny(img, th_1, th_2)

    return img_canny

def blur_and_canny(img):
    blurred = cv.GaussianBlur(img, (7,7), 1)
    canny = cv.Canny(blurred, 120, 255, 1)

    return canny

def crop_image(img: cv.Mat) -> cv.Mat:
    height, width = img.shape

    min_height = int(0.3*height)
    max_height = int(0.85*height)
    min_width = int(0.3*width)

    cropped_image = img[min_height:max_height, min_width:width]

    return cropped_image

def resize_image(img: cv.Mat, width=1280, height=720) -> cv.Mat:
    resized = cv.resize(img, (width, height))
    
    return resized

In [ ]:
img_original = cv.imread("../images/dungeon2.png")
img_original = cv.cvtColor(img_original, cv.COLOR_BGR2RGB)
img = cv.cvtColor(img_original, cv.COLOR_RGB2GRAY)

img = crop_image(img)
img = preprocess.blur(img, (1,1))
ret, img = cv.threshold(img, 55, 255, cv.THRESH_BINARY)
img = canny(img)
contours, hierarchy = cv.findContours(img, cv.RETR_CCOMP, cv.CHAIN_APPROX_SIMPLE)

cv.drawContours(img_original, contours, -1, (255,0,0), 1)
approxes = []
for i, cnt in enumerate(contours):
    epsilon = 0.05*cv.arcLength(cnt,True)
    approx = cv.approxPolyDP(cnt,epsilon,True)
    if len(approx) == 3:
        approxes.append(cnt)
        cv.drawContours(img_original, [cnt], 0, (0,255,0), 2)
print(len(approxes))

plt.figure(figsize=(16, 8))
plt.imshow(img_original)

In [ ]:
from dataclasses import dataclass

@dataclass
class Nodes():
    x_start: int = 700
    y_start: int = 380
    width: int = 60
    height: int = 60
    x_stride: int = 160
    y_stride: int = 130

@dataclass
class Node():
    name: str
    x: int 
    y: int 
    width: int 
    height: int 
    connection: list["Node"]



In [ ]:
x_start = 690
y_start = 370

width = 80
height = 80

x_stride = 160
y_stride = 130

param = Nodes()

nodes = []
index = 1
for i in range(4):
    for j in range(3):
        name = f"Node_{index}"
        node = Node(name, x_start, y_start, width, height, connection=[])
        nodes.append(node)

        y_start = y_start + y_stride
        index = index + 1
    y_start = 370
    x_start = x_start + x_stride

In [ ]:
img_original = cv.imread("../images/train_1.png")
img_original = cv.cvtColor(img_original, cv.COLOR_BGR2RGB)
# img = img_original
img = cv.cvtColor(img_original, cv.COLOR_RGB2GRAY)

# img = cv.add(img, 50)
# img = preprocess.blur(img, (3,3))
# ret, img = cv.threshold(img, 55, 255, cv.THRESH_BINARY)
# img = canny(img)

fig, axes = plt.subplots(3, 4, figsize=(16,8))
index = 0
for i in range(4):
    for j in range(3):
        node = nodes[index]
        axes[j,i].set_title(node.name)
        axes[j,i].imshow(img[node.y:node.y+node.height, node.x:node.x+node.width], cmap='gray')
        index = index + 1
        
plt.show()

In [ ]:
from feature_detection import detect_feature, match_feature
import os

dir = "D:\Repository\python-limbus\images\encounter"
queries = []
predictions = []
for file in os.listdir(dir):
    q = cv.imread(f"{dir}\\{file}", cv.IMREAD_GRAYSCALE)
    kp, desc = detect_feature(q, edge_threshold=10)
    query = {
        "name": file.split(".")[0],
        "img": q,
        "keypoints": kp,
        "descriptor": desc
    }
    queries.append(query)

for i, node in enumerate(nodes):
    node_location = img[node.y:node.y+node.height, node.x:node.x+node.width]
    kp, desc = detect_feature(node_location, edge_threshold=10)
    print(i+1)

    if desc is None: 
        continue
    
    pred = {"name":"", "min":0}
    for query in queries:
        matches = match_feature(query["descriptor"], desc, True)
        top_ten = [match.distance for match in matches[:10]]

        if pred["min"] == 0:
            pred["name"] = query["name"]
            pred["min"] = mean(top_ten)
        elif pred["min"] > mean(top_ten):
            pred["name"] = query["name"]
            pred["min"] = mean(top_ten)

    print(pred["name"])



In [ ]:
img1 = queries[3]

node = nodes[-1]
img2 = img[node.y:node.y+node.height, node.x:node.x+node.width]
kp2, desc2 = detect_feature(img2)

matches = match_feature(img1["descriptor"], desc2, True)
print(mean([match.distance for match in matches[:10]]))
img_match = cv.drawMatches(img1["img"], img1["keypoints"], img2, kp2, matches[:10], None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.imshow(img_match)
plt.show()

In [ ]:
img_original = cv.imread("../images/train_1.png")
img_original = cv.cvtColor(img_original, cv.COLOR_BGR2RGB)
img = cv.cvtColor(img_original, cv.COLOR_RGB2GRAY)

plt.figure(figsize=(16, 8))
plt.imshow(img, cmap='gray')

In [ ]:
from limbus.data import Encounters, Config
from limbus.dungeon import Dungeon
import cv2 as cv

dir = "D:\Repository\python-limbus\images\encounter"
dungeon_map = "D:\Repository\python-limbus\images\\train_1.png"
map = cv.imread(dungeon_map, cv.IMREAD_GRAYSCALE)

cfg = Config()
md = Dungeon(map, config=cfg, encounters_dir=dir)

nodes = md.map()
connections = md.map_connections()

connections

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from math import dist

img_dir = "D:\Repository\python-limbus\images\\train_1.png"

img = cv.imread(img_dir, cv.IMREAD_GRAYSCALE)
img = cv.GaussianBlur(img, (3,3), 0)
img = cv.Canny(img, 15, 75)
# img = cv.add(img, 50)

# img_1 = img[360:750, 730:880] # 150
# img_2 = img[360:750, 890:1040] # 150
# img_3 = img[360:750, 1050:1200] # 150

img_1 = img[nodes[0].y:nodes[2].y+nodes[2].height, nodes[0].x+nodes[0].width:nodes[3].x] 
img_2 = img[nodes[0].y:nodes[2].y+nodes[2].height, nodes[3].x+nodes[3].width:nodes[6].x]
img_3 = img[nodes[0].y:nodes[2].y+nodes[2].height, nodes[6].x+nodes[6].width:nodes[9].x]

# rho = 1  # distance resolution in pixels of the Hough grid
# theta = np.pi / 180  # angular resolution in radians of the Hough grid
# threshold = 50  # minimum number of votes (intersections in Hough grid cell)
# min_line_length = 75  # minimum number of pixels making up a line
# max_line_gap = 30  # maximum gap in pixels between connectable line segments

kwargs = {
    "rho": 1,
    "theta": np.pi / 180,
    "threshold": 50,
    "minLineLength": 25,
    "maxLineGap": 50
}

def check_duplicate(index: int, lines: list, min_distance: int=50) -> bool:
    has_duplicate = False
    query = lines[index]

    min_dst = float("inf")
    for i in range(0, index):
        dst = dist(query.reshape(-1), lines[i].reshape(-1))

        if dst < min_dst:
            min_dst = dst
    
    if min_dst < min_distance:
        has_duplicate = True

    return has_duplicate

imgs = [img_1, img_2, img_3]
for img in imgs:
    lines = cv.HoughLinesP(img, **kwargs)
    for i, line in enumerate(lines):
        has_dupe = check_duplicate(i, lines, 50)
        if has_dupe is False:
            for x1,y1,x2,y2 in line:
                cv.line(img, (x1,y1), (x2,y2), (255,0,0), 2)

fig, axes = plt.subplots(1, 3, figsize=(16,8))
for i, img in enumerate(imgs):
    axes[i].imshow(img, cmap='gray')
plt.show()

In [ ]:
print(f"{nodes[0]}")
print(f"{nodes[3]}")
print(f"{nodes[6]}")

for i in range(0,7,3):
    for line in lines:
        for x1, y1, x2, y2 in line:
            x = nodes[i].x + nodes[i].width + x1
            y = nodes[i].y + y1
            xt = nodes[i].x + nodes[i].width + x2
            yt = nodes[i].y + y2
            
            print(f"Position:\t\tActual:")
            print(f"{line}\t\t{[[x, y, xt, yt]]}")